In [1]:
import pyaudio
import wave
import numpy as np
import webrtcvad
import threading
import time


## Voice Recording

In [2]:

class AudioRecorder:
    def __init__(self, output_filename="recorded_audio.wav", max_silence_seconds=3):
        self.output_filename = output_filename
        self.max_silence_seconds = max_silence_seconds
        self.chunk = 320  # 20ms chunks at 16000 Hz
        self.sample_format = pyaudio.paInt16  # 16 bits per sample
        self.channels = 1  # Mono
        self.rate = 16000  # Sample rate suitable for VAD
        self.vad = webrtcvad.Vad()
        self.vad.set_mode(1)  # Mode 1: Low aggressiveness
        self.p = pyaudio.PyAudio()  # Create an interface to PortAudio
        self.stream = None
        self.frames = []
        self.silence_chunks = 0
        self.is_recording = False
        self.recording_thread = None

    def start_recording(self):
        """Start the audio recording in a separate thread."""
        if self.is_recording:
            print("Recording is already in progress.")
            return

        self.is_recording = True
        self.frames = []  # Reset frames
        self.silence_chunks = 0  # Reset silence counter

        # Open stream
        try:
            self.stream = self.p.open(format=self.sample_format,
                                      channels=self.channels,
                                      rate=self.rate,
                                      frames_per_buffer=self.chunk,
                                      input=True)
        except Exception as e:
            print(f"Error opening stream: {e}")
            self.is_recording = False
            return

        print("Recording started...")
        self.recording_thread = threading.Thread(target=self._record)
        self.recording_thread.start()

    def _record(self):
        """Internal method to handle the recording process."""
        try:
            while self.is_recording:
                try:
                    data = self.stream.read(self.chunk, exception_on_overflow=False)
                except Exception as e:
                    print(f"Error reading stream: {e}")
                    break

                self.frames.append(data)

                # Convert data to numpy array for VAD processing
                audio_data = np.frombuffer(data, dtype=np.int16)

                # Ensure the audio chunk is of the correct length for VAD processing
                if len(audio_data) == self.chunk:
                    is_speech = self.vad.is_speech(audio_data.tobytes(), sample_rate=self.rate)
                else:
                    is_speech = False

                # Check for silence
                if not is_speech:
                    self.silence_chunks += 1
                else:
                    self.silence_chunks = 0  # Reset if speech is detected

                # Stop recording if silence is detected for more than max_silence_seconds
                if self.silence_chunks > (self.max_silence_seconds * self.rate / self.chunk):
                    print("Silence detected. Stopping recording.")
                    self.stop_recording()

        except Exception as e:
            print(f"Error during recording: {e}")
            self.stop_recording()

    def stop_recording(self):
        """Stop the recording and save the audio file."""
        if not self.is_recording:
            print("Recording is not active.")
            return

        # Safely stop recording
        self.is_recording = False

        # Stop and close the stream safely
        if self.stream is not None:
            try:
                self.stream.stop_stream()
                self.stream.close()
            except Exception as e:
                print(f"Error closing stream: {e}")

        # Terminate the PortAudio interface safely
        try:
            self.p.terminate()
        except Exception as e:
            print(f"Error terminating PortAudio: {e}")

        # Save the recorded data as a WAV file only if there are frames
        if self.frames:
            try:
                with wave.open(self.output_filename, 'wb') as wf:
                    wf.setnchannels(self.channels)
                    wf.setsampwidth(self.p.get_sample_size(self.sample_format))
                    wf.setframerate(self.rate)
                    wf.writeframes(b''.join(self.frames))
                print(f"Recording finished. File saved as {self.output_filename}")
            except Exception as e:
                print(f"Error saving WAV file: {e}")
        else:
            print("No audio frames recorded, nothing to save.")




In [3]:
recorder = AudioRecorder()


#### Start Recording

In [4]:
# Start recording
recorder.start_recording()

Recording started...


#### Stop Recording

In [5]:
# Simulating a manual stop after some time, replace this with button control in your front end
recorder.stop_recording()

Recording finished. File saved as recorded_audio.wav
